In [ ]:
from elasticsearch import Elasticsearch
from datetime import datetime
import json
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Elasticsearch 클라이언트 초기화
es = Elasticsearch("http://localhost:9200")  # Elasticsearch 서버 주소

# 산림 코드 매핑
forest_type_map = {
    "C": "침엽수림", "H": "활엽수림", "M": "침활혼효림", "D": "소나무림",
    "PD": "소나무 인공림", "PK": "잣나무림", "PL": "낙엽송림", "PR": "리기다소나무림",
    "Q": "참나무림", "PQ": "참나무인공림", "Po": "포푸라림", "Ca": "밤나무림",
    "Cr": "삼나무림", "Co": "편백나무림", "Ab": "전나무", "Pc": "침엽수 인공림",
    "PH": "활엽수 인공림", "B": "죽림", "F": "별채적지", "O": "이립목지",
    "E": "황폐지", "LP": "목장", "L": "경작지", "R": "제지", "W": "수체",
    "Dn": "임간나지", "99": "해당없음"
}

# 인덱스 매핑 생성
index_name = "real_mountain_fire"
mapping = {
    "mappings": {
        "properties": {
            "지번주소": {"type": "keyword"},
            "사유": {"type": "keyword"},
            "피해면적": {"type": "double"},
            "피해금액": {"type": "double"},
            "시/도명": {"type": "keyword"},
            "발생시각": {"type": "date"},
            "진화시간": {"type": "integer"},
            "FIFTH_FRTP": {"type": "keyword"},
            "산림명": {"type": "keyword"},
            "location": {"type": "geo_point"}
        }
    }
}

# 인덱스 생성
if not es.indices.exists(index=index_name):
    es.indices.create(index=index_name, body=mapping)

# GeoJSONL 파일 읽기 및 데이터 전송
file_path = "C:/workspace/3rd_project/_data/geojsonl/real_mountain_fire.geojsonl"

with open(file_path, "r", encoding="utf-8") as file:
    for line in file:
        feature = json.loads(line)
        props = feature["properties"]

        # location 필드 생성
        props["location"] = {
            "lat": props["위도"],
            "lon": props["경도"]
        }
        # 산림명 매핑
        props["산림명"] = forest_type_map.get(props["FIFTH_FRTP"], "알 수 없음")
        # 불필요한 필드 제거
        del props["위도"], props["경도"]

        # 데이터 인덱싱
        es.index(index=index_name, body=props)

print("GeoJSONL 데이터 전송 완료")
